# 1.0 Setup

## 1.1 Package Imports

In [ ]:
import geopandas as gpd
import pandas as pd
import time
from shapely.geometry import Point, Polygon, MultiPolygon
import osmnx as ox
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

## 1.2 Data Import

In [ ]:
# Setting the file path
path = r'YOUR FILE PATH'

In [ ]:
# Reading in the data as a Pandas DataFrame
#REMINDER - The listings data must be downloaded from Inside Airbnb
listings = pd.read_csv(path + r'NY Airbnb June 2020\listings.csv.gz', compression='gzip', low_memory=False)

# Converting it to a GeoPandas DataFrame
listings_gpdf = gpd.GeoDataFrame(
    listings,
    geometry=gpd.points_from_xy(listings['longitude'],
                                   listings['latitude'],
                                   crs="EPSG:4326")
)

# Printing the shape of the DataFrame
listings_gpdf.shape

In [ ]:
# Focusing on attractions in Manhattan, so we need to create a mask to filter locations 
# in the Manhattan borough
boroughs = gpd.read_file(path + r"NYC Boroughs\nybb_22a\nybb.shp")
boroughs = boroughs.to_crs('EPSG:4326')
manhattan = boroughs[boroughs['BoroName']=='Manhattan']

# get the start time
st = time.time()

listings_mask = listings_gpdf.within(manhattan.loc[3, 'geometry'])

listings_manhattan = listings_gpdf.loc[listings_mask]

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

In [ ]:
listings_manhattan.shape

# 2.0 Spatial Indexing

## 2.1 R-Tree

In [ ]:
# Building the R-tree spatial index
sindex = listings_gpdf.sindex

# get the start time
st = time.time()

# Getting the indexes of the possible matches from the R-tree
idex_possible_matches = list(sindex.intersection(geometry.bounds))

# subsetting the dataframe to be only possible matches
possible_matches_df = listings_gpdf.iloc[idex_possible_matches]

# Performing an intersection to get the precise matches
precise_matches_df = possible_matches_df[possible_matches_df.intersects(geometry)]

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

In [ ]:
precise_matches.shape

### 2.1.1 R-Tree with Subdivided Polygons

In [ ]:
# Idenitifying units of measurement
listings_gpdf.crs.axis_info

In [ ]:
# Create the subdivided polygons
subdivided_polygon = ox.utils_geo._quadrat_cut_geometry(geometry, quadrat_width=1) # quadrant_width is in the CRS measurement units (4326:degrees)

In [ ]:
# get the start time
st = time.time()

points_in_geometry = pd.DataFrame()
for geom in subdivided_polygon:
    # add in a slight buffer to account for points falling on the lines of the subdivided polygons
    geom = geom.buffer(1e-14).buffer(0)

    # Getting the indexes of the possible matches from the R-tree
    idex_possible_matches = list(sindex.intersection(geom.bounds))
    possible_matches_df = listings_gpdf.iloc[idex_possible_matches]
    
    # Performing an intersection to get the precise matches
    precise_matches_df = possible_matches_df[possible_matches_df.intersects(geom)]
    points_in_geometry = points_in_geometry.append(precise_matches_df)

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

In [ ]:
points_in_geometry.shape

## 2.2 H3

In [ ]:
# Reference Materials
# https://www.uber.com/blog/h3/
# https://spatialthoughts.com/2020/07/01/point-in-polygon-h3-geopandas/

In [ ]:
%matplotlib inline 
from h3 import h3
import contextily

In [ ]:
# Set the H3 resolution
h3_resolution = 8
 
# Creating a function to add the H3 identifier to each of the Airbnb Points
def add_h3_id(row):
    return h3.geo_to_h3(
      row.geometry.y, row.geometry.x, h3_level)

# Executing the function
listings_manhattan['h3'] = listings_gpdf.apply(add_h3_id, axis=1)

# Display the dataframe
listings_manhattan.head()

In [ ]:
# Creating a dataframe with the count of airbnb's within each hexagon
airbnb_count = listings_manhattan.groupby(['h3']).h3.agg('count').to_frame('count').reset_index()

In [ ]:
# Defining a function to get the geometry for each of the H3 hexagons
def add_h3_geometry(row):
    points = h3.h3_to_geo_boundary(
      row['h3'], True)
    return Polygon(points)

# Adding the geometry to the airbnb_count dataframe
airbnb_count['geometry'] = airbnb_count.apply(add_geometry, axis=1)

# Converting to a geodataframe
gdf = gpd.GeoDataFrame(counts, crs='EPSG:4326')

In [ ]:
# Plotting a choropleth map of the Airbnb's within each cell
f, ax = plt.subplots(1, figsize=(10, 20))
   
# Plot choropleth of counts
gdf.plot(
    column='count', 
    cmap='coolwarm', 
    scheme='quantiles',
    k=4, 
    edgecolor='white', 
    linewidth=0.1, 
    alpha=0.5,
    legend=True,
    ax=ax
)

# Add basemap
contextily.add_basemap(
    ax,
    crs=gdf.crs,
    source=contextily.providers.Stamen.TonerLite,
)
    
# Remove axis
ax.set_axis_off()

# Display
plt.show()
#f.savefig(path + r"Neighborhood Residuals.png")